In [3]:
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import tensorflow as tf
import os
import zipfile
import shutil
import random
from shutil import copyfile

In [4]:
    def convert_to_ela_image(image):
        quality=90;
        #im=image.convert('RGB')
        im = Image.fromarray(image,'RGB')
        im.save('resaved.jpg','JPEG', quality=quality)
        resaved_im=Image.open('resaved.jpg')
        ela_im = ImageChops.difference(im, resaved_im)
        os.remove('resaved.jpg')
        extrema = ela_im.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        scale = 255.0/max_diff 

        ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
        return np.array(ela_im,dtype='float64')

In [20]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5),padding='valid',activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.Conv2D(32,(5,5),strides=(2,2),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D((2,2),strides=None,padding='valid',data_format='channels_last'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
]);
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 60, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               7373056   
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [21]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.01,rho=0.9, epsilon=1e-08, decay=0.0),loss='categorical_crossentropy',metrics=['accuracy']);

In [19]:
local_zip='data.zip';
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('Data')
zip_ref.close()

In [8]:
print(len(os.listdir('Data/data')))

2001


In [9]:
os.mkdir('data2');
os.mkdir('data2/tamp');
os.mkdir('data2/scale');

In [10]:
names=os.listdir('Data/data');
for i in range(0,len(names)):
    file=names[i];
    path='Data/data/'+file;
    if(file.find('tamp') != -1):
        shutil.move(path,'data2/tamp/');
    else:
        shutil.move(path,'data2/scale/');

In [11]:
print(len(os.listdir('data2/tamp')))
print(len(os.listdir('data2/scale')))

700
1301


In [12]:
os.mkdir('data3');
os.mkdir('data3/training');
os.mkdir('data3/testing');
os.mkdir('data3/training/tamp');
os.mkdir('data3/training/scale');
os.mkdir('data3/testing/tamp');
os.mkdir('data3/testing/scale');

In [13]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    listing=os.listdir(SOURCE);
    random.sample(listing,len(listing));
    list2=listing[0:int(len(listing)-SPLIT_SIZE*len(listing))];
    list3=list(set(listing)-set(list2));
    c=0;
    for i in list2:
        if(os.path.getsize(SOURCE+i)>0):
            copyfile(SOURCE+i,TRAINING+i);
    for i in list3:
        if(os.path.getsize(SOURCE+i)>0):
            copyfile(SOURCE+i,TESTING+i);

tamp_SOURCE_DIR = 'data2/tamp/'
TRAINING_tamp_DIR = 'data3/training/tamp/'
TESTING_tamp_DIR = 'data3/testing/tamp/'
scale_SOURCE_DIR = 'data2/scale/'
TRAINING_scale_DIR = 'data3/training/scale/'
TESTING_scale_DIR = 'data3/testing/scale/'
split_size = .1
split_data(tamp_SOURCE_DIR, TRAINING_tamp_DIR, TESTING_tamp_DIR, split_size)
split_data(scale_SOURCE_DIR, TRAINING_scale_DIR, TESTING_scale_DIR, split_size)

In [14]:
print(len(os.listdir('data3/training/scale')))
print(len(os.listdir('data3/testing/scale')))

1170
131


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = 'data3/training'
train_datagen = ImageDataGenerator(
      preprocessing_function=convert_to_ela_image,
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,target_size=(128,128),batch_size=5,class_mode='categorical')
VALIDATION_DIR = 'data3/testing'
validation_datagen = ImageDataGenerator(rescale=1.0/255.0);
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,target_size=(128,128),batch_size=10,class_mode='categorical');

Found 1799 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=359,  # 2000 images = batch_size * steps
      epochs=12,
      validation_data=validation_generator,
      validation_steps=20)

Epoch 1/12
211/359 [================>.............] - ETA: 1:09 - loss: 10.7389 - accuracy: 0.6480